This notebook handles the interaction with the Github API.

The notebook will *only* run in the JupyterLite environment.

In [ ]:
%pip install ipywidgets

In [ ]:
import json
from js import Object, fetch
import ipywidgets
from IPython import display
# from pyodide.ffi import to_js

The two text widgets will ask you for the owner and name of the repository you want to get the commits for.

Example: For `https://github.com/hpi-swt2/bookkeeper-portal-red` the owner is `hpi-swt2` and the name is `bookkeeper-portal-red`.

In [ ]:
owner = ipywidgets.Text(
    value="hpi-swt2",
    placeholder="Enter the name of the repository owner",
    description="Repo owner:",
    disabled=False
)

repository = ipywidgets.Text(
    value="bookkeeper-portal-red",
    placeholder="Enter the name of your repository",
    description="Repo name:",
    disabled=False
)

# auth_token = ipywidgets.Text(
#     value="YOUR-TOKEN",
#     placeholder="Enter your GitHub auth token",
#     description="Auth token:",
#     disabled=False
# )

display.display(owner)
display.display(repository)
# display.display(auth_token)

This cell gets a list of all commits for the given repository and saves the data to a file.

You will notice the commented out headers argument.
We would normally use this to authenticate ourselves against the Github API, but unfortunately, the `fetch` call breaks with an uninformative error message when we add these headers.

The consequence of this is that all requests are subject to a **lower rate limit**, so make sure to not run this cell too often.

In [ ]:
page = 0
commits = []

while True:
    resp = await fetch(f"https://api.github.com/repos/{owner.value}/{repository.value}/commits?per_page=100&page={page}", 
                       method="GET",
                       # headers=Object.fromEntries(to_js(
                       #     { 
                       #      "Accept": "application/vnd.github+json", 
                       #      "Authorization": f"Bearer {auth_token.value}",
                       #      "X-GitHub-Api-Version": "2022-11-28"
                       #     }
                       # ))
                      )
    res = await resp.text()
    res = json.loads(res)

    # All commits were fetched
    if len(res) == 0:
        break
        
    # Rate limit exceeded
    if len(res) == 2 and res["message"].startswith("API rate limit exceeded"):
        print(res["message"])
        break
    
    commits.extend(res)
    page += 1   

# Write the commit data to a JSON file
with open(f"commits_{owner.value}_{repository.value}.json", "w") as f:
    json.dump(commits, f, indent=4)